In [58]:
import wandb
import matplotlib.pyplot as plt
import dice_ml
import warnings
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from dice_ml.explainer_interfaces.explainer_base import UserConfigValidationException
import plotly.graph_objects as go
from datetime import timedelta

from wandb.env import DATA_DIR

from scripts.SEP_CFE_functions import *
from slim_tsf.func._slim_TSF import SlimTimeSeriesForestClassifier
from aeon.utils.numba.stats import row_mean
import re
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import dice_ml
from sklearn.model_selection import train_test_split
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer, roc_auc_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from tqdm import tqdm

import seaborn as sns
import joblib
import os
from sklearn.preprocessing import MinMaxScaler
from scripts.SEP_CFE_functions import *
min_max_scaler = MinMaxScaler()
warnings.filterwarnings("ignore")

In [59]:
DATA_DIR = "/Users/pranjal/PycharmProjects/SolarEnergyParticlePrediction/data/processed/"
DATA_NPZ = DATA_DIR+"sep_slim_tsf_input_dataset.npz"
RNG = 42
SAVE_DIR = "/Users/pranjal/PycharmProjects/SolarEnergyParticlePrediction/models/classifiers_20feat/"
os.makedirs(SAVE_DIR, exist_ok=True)

In [60]:
train_df = pd.read_csv(DATA_DIR + "train_All_window_included_reduced_top20_rowmean_dataset.csv", sep=',')
test_df = pd.read_csv(DATA_DIR + "test_All_window_included_reduced_top20_rowmean_dataset.csv", sep=',')
val_df = pd.read_csv(DATA_DIR + "val_All_window_included_reduced_top20_rowmean_dataset.csv", sep=',')
combined_df = pd.read_csv(DATA_DIR + "combined_All_window_included_reduced_top20_rowmean_with_filelist.csv", sep=',')


In [61]:
X_train = train_df.drop(columns=['label'])
y_train = train_df['label']
X_test = test_df.drop(columns=['label'])
y_test = test_df['label']
X_val = val_df.drop(columns=['label'])
y_val = val_df['label']

In [82]:
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (1416, 20) (1416,)
Validation set shape: (608, 20) (608,)
Test set shape: (868, 20) (868,)


In [62]:
# data = np.load(DATA_NPZ)
# X, y = data["X"], data["y"]
#
# train_x, test_x, train_y, test_y = train_test_split(
#     X, y, test_size=0.3, stratify=y, random_state=RNG
# )
# train_x, val_x, train_y, val_y = train_test_split(
#     train_x, train_y, test_size=0.3, stratify=train_y, random_state=RNG
# )

In [63]:
# print("Training set shape:", train_x.shape, train_y.shape)
# print("Validation set shape:", val_x.shape, val_y.shape)
# print("Test set shape:", test_x.shape, test_y.shape)

In [64]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score, pos_label=1)


In [65]:
# from sklearn.linear_model import LogisticRegression
#
# log_reg = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf', LogisticRegression(max_iter=5000, class_weight='balanced', solver='saga'))
# ])
#
# param_grid_logreg = {
#     'clf__penalty': ['l1', 'l2', 'elasticnet'],
#     'clf__C': np.logspace(-3, 3, 7),
#     'clf__l1_ratio': [0.0, 0.25, 0.5, 0.75, 1.0],
# }
#
# grid_logreg = GridSearchCV(
#     estimator=log_reg,
#     param_grid=param_grid_logreg,
#     scoring=scorer,
#     cv=cv,
#     n_jobs=-1,
#     verbose=2
# )
#
# grid_logreg.fit(X_train, y_train)
# best_logreg = grid_logreg.best_estimator_
#
# print("Best Logistic Regression params:", grid_logreg.best_params_)
# print("Best F1 score:", grid_logreg.best_score_)
#
# # SAVE MODEL
# joblib.dump(best_logreg, os.path.join(SAVE_DIR, "best_logistic_reg.pkl"))


Fitting 5 folds for each of 105 candidates, totalling 525 fits


/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elastic

[CV] END ...clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=l1; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=l1; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=l2; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=l2; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=l2; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=l1; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=0.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END ..clf__C=0.001, clf__l1_ratio=0.25, clf__penalty=l1; total time=   0.0s
[CV

/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1175: UserWarning: l1_ratio parameter is only used when penalty is 'elastic

[CV] END ...clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END ..clf__C=0.001, clf__l1_ratio=0.75, clf__penalty=l2; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END ..clf__C=0.001, clf__l1_ratio=0.75, clf__penalty=l2; total time=   0.0s
[CV] END clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.0s
[CV] END ...clf__C=0.001, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.0s
[CV] END ....clf__C=0.01, clf__l1_ratio=0.0, clf__penalty=l2; total time=   0.0s
[CV] END

...

[CV] END ..clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=l1; total time=   1.0s
[CV] END ..clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=l1; total time=   1.0s
[CV] END ..clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=l1; total time=   1.1s
[CV] END ..clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.5s
[CV] END ..clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.7s
[CV] END ..clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.8s
[CV] END ..clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.8s


/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END ..clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=l2; total time=   0.7s
[CV] END clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.5s
[CV] END clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.8s
[CV] END clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.8s
[CV] END clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.8s
[CV] END clf__C=1000.0, clf__l1_ratio=1.0, clf__penalty=elasticnet; total time=   0.8s
Best Logistic Regression params: {'clf__C': 0.001, 'clf__l1_ratio': 0.0, 'clf__penalty': 'l2'}
Best F1 score: 0.7007330705874939


/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/pranjal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


['/Users/pranjal/PycharmProjects/SolarEnergyParticlePrediction/models/classifiers_20feat/best_logistic_reg.pkl']

In [66]:
# from sklearn.linear_model import RidgeClassifier
#
# ridge = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf', RidgeClassifier(class_weight='balanced'))
# ])
#
# param_grid_ridge = {
#     'clf__alpha': np.logspace(-3, 3, 7)
# }
#
# grid_ridge = GridSearchCV(
#     estimator=ridge,
#     param_grid=param_grid_ridge,
#     scoring=scorer,
#     cv=cv,
#     n_jobs=-1,
#     verbose=2
# )
#
# grid_ridge.fit(X_train, y_train)
# best_ridge = grid_ridge.best_estimator_
#
# print("Best Ridge params:", grid_ridge.best_params_)
# print("Best F1 score:", grid_ridge.best_score_)
#
# # SAVE MODEL
# joblib.dump(best_ridge, os.path.join(SAVE_DIR, "best_ridge_classifier.pkl"))


Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV] END ...................................clf__alpha=0.001; total time=   0.0s
[CV] END ...................................clf__alpha=0.001; total time=   0.0s
[CV] END ...................................clf__alpha=0.001; total time=   0.0s
[CV] END ...................................clf__alpha=0.001; total time=   0.0s
[CV] END ...................................clf__alpha=0.001; total time=   0.0s
[CV] END ....................................clf__alpha=0.01; total time=   0.0s
[CV] END ....................................clf__alpha=0.01; total time=   0.0s
[CV] END ....................................clf__alpha=0.01; total time=   0.0s
[CV] END ....................................clf__alpha=0.01; total time=   0.0s
[CV] END ....................................clf__alpha=0.01; total time=   0.0s
[CV] END .....................................clf__alpha=0.1; total time=   0.0s
[CV] END .....................................clf

['/Users/pranjal/PycharmProjects/SolarEnergyParticlePrediction/models/classifiers_20feat/best_ridge_classifier.pkl']

In [67]:
# from sklearn.svm import SVC
#
# svm_rbf = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf', SVC(kernel='rbf', probability=True, class_weight='balanced'))
# ])
#
# param_grid_svm = {
#     'clf__C': np.logspace(-3, 3, 7),
#     'clf__gamma': ['scale', 'auto'] + list(np.logspace(-4, 1, 6)),
# }
#
# grid_svm = GridSearchCV(
#     estimator=svm_rbf,
#     param_grid=param_grid_svm,
#     scoring=scorer,
#     cv=cv,
#     n_jobs=-1,
#     verbose=2
# )
#
# grid_svm.fit(X_train, y_train)
# best_svm = grid_svm.best_estimator_
#
# print("Best SVM params:", grid_svm.best_params_)
# print("Best F1 score:", grid_svm.best_score_)
#
# # SAVE MODEL
# joblib.dump(best_svm, os.path.join(SAVE_DIR, "best_svm_rbf.pkl"))


Fitting 5 folds for each of 56 candidates, totalling 280 fits
[CV] END .....................clf__C=0.001, clf__gamma=scale; total time=   0.2s
[CV] END ......................clf__C=0.001, clf__gamma=auto; total time=   0.2s
[CV] END ......................clf__C=0.001, clf__gamma=auto; total time=   0.2s
[CV] END ......................clf__C=0.001, clf__gamma=auto; total time=   0.2s
[CV] END .....................clf__C=0.001, clf__gamma=scale; total time=   0.2s
[CV] END .....................clf__C=0.001, clf__gamma=scale; total time=   0.2s
[CV] END .....................clf__C=0.001, clf__gamma=scale; total time=   0.2s
[CV] END .....................clf__C=0.001, clf__gamma=scale; total time=   0.3s
[CV] END ......................clf__C=0.001, clf__gamma=auto; total time=   0.2s
[CV] END ......................clf__C=0.001, clf__gamma=auto; total time=   0.2s
[CV] END ....................clf__C=0.001, clf__gamma=0.0001; total time=   0.2s
[CV] END ....................clf__C=0.001, clf_

['/Users/pranjal/PycharmProjects/SolarEnergyParticlePrediction/models/classifiers_20feat/best_svm_rbf.pkl']

In [68]:
# import re
#
# # Clean column names
# X_train_xgb = X_train.copy()
# X_train_xgb.columns = [
#     re.sub(r"[\[\]\(\)\<\>]", "_", c).replace(":", "_")
#     for c in X_train_xgb.columns
# ]
#
# X_test_xgb = X_test.copy()
# X_test_xgb.columns = [
#     re.sub(r"[\[\]\(\)\<\>]", "_", c).replace(":", "_")
#     for c in X_test_xgb.columns
# ]
#
# X_val_xgb = X_val.copy()
# X_val_xgb.columns = [
#     re.sub(r"[\[\]\(\)\<\>]", "_", c).replace(":", "_")
#     for c in X_val_xgb.columns
# ]


In [69]:
# from xgboost import XGBClassifier
#
# pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
#
# xgb = XGBClassifier(
#     objective='binary:logistic',
#     eval_metric='logloss',
#     subsample=0.8,
#     colsample_bytree=0.8,
#     use_label_encoder=False,
#     scale_pos_weight=pos_weight,
#     tree_method="hist",   # fast and good for tabular data
#     enable_categorical=False
# )
#
# param_grid_xgb = {
#     'n_estimators': [100, 300, 500],
#     'max_depth': [2, 3, 4, 5],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'gamma': [0, 0.1, 0.3, 0.5],
# }
#
# grid_xgb = GridSearchCV(
#     estimator=xgb,
#     param_grid=param_grid_xgb,
#     scoring=scorer,
#     cv=cv,
#     n_jobs=1,
#     verbose=2
# )
#
# grid_xgb.fit(X_train_xgb, y_train)
# best_xgb = grid_xgb.best_estimator_
#
# print("Best XGBoost params:", grid_xgb.best_params_)
# print("Best F1 score:", grid_xgb.best_score_)
#
# # SAVE MODEL
# joblib.dump(best_xgb, os.path.join(SAVE_DIR, "best_xgboost.pkl"))


Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_estimators=300; total time=   0.1s
[CV] END gamma=0, learning_rate=0.01, max_depth=2, n_esti

KeyboardInterrupt: 

In [70]:
paths = {
    "logreg": SAVE_DIR+"best_logistic_reg.pkl",
    "ridge": SAVE_DIR+"best_ridge_classifier.pkl",
    "svm_rbf": SAVE_DIR+"best_svm_rbf.pkl"
    # "xgboost": SAVE_DIR+"best_xgboost.pkl"
}

models = {name: joblib.load(p) for name, p in paths.items()}

In [71]:
class DiCEModelWrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, X):
        return self.model.predict(X)

    def predict_proba(self, X):
        try:
            return self.model.predict_proba(X)
        except:
            decision = self.model.decision_function(X)
            prob1 = 1 / (1 + np.exp(-decision))
            prob0 = 1 - prob1
            return np.column_stack([prob0, prob1])


In [72]:
wrapped_models = {
    name: DiCEModelWrapper(m) for name, m in models.items()
}


In [73]:
df_dice = X_train.copy()
df_dice["label"] = y_train

continuous_feats = list(X_train.columns)

dice_data = dice_ml.Data(
    dataframe=df_dice,
    continuous_features=continuous_feats,
    outcome_name="label"
)


In [74]:
def build_dice_explainer(model):
    return dice_ml.Dice(
        data_interface=dice_data,
        model_interface=dice_ml.Model(model=model,
                                      backend="sklearn",
                                      model_type="classifier"),
        method="genetic"
    )


In [75]:
explainers = {
    name: build_dice_explainer(wrapped_models[name])
    for name in wrapped_models
}


In [76]:
def validity(model, x_cf, desired):
    pred = model.predict(x_cf)[0]
    return int(pred == desired)

def proximity(x_orig, x_cf):
    return np.linalg.norm(x_orig - x_cf, ord=2)

def sparsity(x_orig, x_cf):
    return np.sum(np.abs(x_orig - x_cf) > 1e-6)

def confidence_gain(model, x_orig, x_cf, desired):
    p_orig = model.predict_proba(x_orig)[0][desired]
    p_cf   = model.predict_proba(x_cf)[0][desired]
    return p_cf - p_orig


In [77]:
def generate_cfe(explainer, x_query, target):
    try:
        cfe = explainer.generate_counterfactuals(
            x_query,
            total_CFs=1,
            desired_class=target
        )
        return cfe.cf_examples_list[0].final_cfs_df
    except:
        return None


In [78]:
def evaluate_fidelity_on_test(model_name, model, explainer,
                              X_test, y_test, max_samples=None):

    results = []

    n_samples = len(X_test) if max_samples is None else min(max_samples, len(X_test))

    for idx in tqdm(range(n_samples), desc=f"Evaluating {model_name}"):

        x_query = X_test.iloc[[idx]]
        y_pred = model.predict(x_query)[0]
        y_target = 1 - y_pred

        # If already in target class → skip
        if y_pred == y_target:
            continue

        # Generate CF
        cf_df = generate_cfe(explainer, x_query, y_target)
        if cf_df is None:
            # CF failed
            results.append({
                "model": model_name,
                "idx": idx,
                "validity": 0,
                "proximity": np.nan,
                "sparsity": np.nan,
                "confidence_gain": np.nan
            })
            continue

        x_cf = cf_df.to_numpy()
        x_orig = x_query.to_numpy()

        results.append({
            "model": model_name,
            "idx": idx,
            "validity": validity(model, x_cf, y_target),
            "proximity": proximity(x_orig, x_cf),
            "sparsity": sparsity(x_orig, x_cf),
            "confidence_gain": confidence_gain(model, x_query, x_cf, y_target)
        })

    return pd.DataFrame(results)


In [88]:
full_results = []

for name in wrapped_models:
    res = evaluate_fidelity_on_test(
        model_name=name,
        model=wrapped_models[name],
        explainer=explainers[name],
        X_test=X_test,
        y_test=y_test,
        max_samples=None      # set e.g. 200 for quick tests
    )
    full_results.append(res)

df_fidelity = pd.concat(full_results, ignore_index=True)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:

In [85]:
summary = df_fidelity.groupby("model").agg({
    "validity": "mean",
    "proximity": "mean",
    "sparsity": "mean",
    "confidence_gain": "mean"
})
print(summary)


         validity  proximity  sparsity  confidence_gain
model                                                  
logreg        0.0        NaN       NaN              NaN
ridge         0.0        NaN       NaN              NaN
svm_rbf       0.0        NaN       NaN              NaN
